In [ ]:
!gdown --id 1Rb-9DYp8hHO60Tk_-_kK1E7Axeaog4rA # dataset

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Rb-9DYp8hHO60Tk_-_kK1E7Axeaog4rA
To: /content/segthor png.7z
100% 771M/771M [00:05<00:00, 143MB/s]


In [ ]:
!gdown --id 1XA3RD_sKtcNitf-3SQ_a85_v9Q2_DU3r # resnet weights

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1XA3RD_sKtcNitf-3SQ_a85_v9Q2_DU3r
To: /content/resnet18_v2.pth
100% 44.8M/44.8M [00:01<00:00, 41.4MB/s]


In [ ]:
!7z x '/content/segthor png.7z' -o"/content/segthor"


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 771039890 bytes (736 MiB)

Extracting archive: /content/segthor png.7z
 11% 4096 Open              WARNING:
/content/segthor png.7z
Can not open the file as [7z] archive
The file is open as [zip] archive

--
Path = /content/segthor png.7z
Open WARNING: Can not open the file as [7z] archive
Type = zip
Physical Size = 771039890

  0%      1% 210 - segthor png/Patient_01_image_287.png                                                 1% 380 - segthor png/Patient_01_image_44.png                                

In [ ]:
!gdown --id 1M-rXJOanrC7kPhcjHq5YSdkRRmzCng0s

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1M-rXJOanrC7kPhcjHq5YSdkRRmzCng0s
To: /content/swin_unet20epochs.pth
100% 168M/168M [00:05<00:00, 30.8MB/s]


In [ ]:
!pip install -q segmentation-models-pytorch transformers

     |████████████████████████████████| 88 kB 6.7 MB/s 
     |████████████████████████████████| 4.2 MB 65.1 MB/s 
     |████████████████████████████████| 58 kB 7.2 MB/s 
     |████████████████████████████████| 376 kB 62.2 MB/s 
     |████████████████████████████████| 6.6 MB 61.4 MB/s 
     |████████████████████████████████| 596 kB 71.6 MB/s 
     |████████████████████████████████| 84 kB 3.8 MB/s 


In [ ]:
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from torchvision import transforms
import torchvision.transforms.functional as TF
import torch.optim as optim
from time import time
from PIL import Image
import matplotlib.pyplot as plt
import glob
import os
import numpy as np
from IPython.display import clear_output
import time
import pandas as pd

class Patient(Dataset):
    def __init__(self, csv_root, png_root, patient_ID):
        self.csv = pd.read_csv(csv_root)

        # select only patient_ID
        self.csv = self.csv[self.csv['patient_ID'] == patient_ID]
        self.csv = self.csv.reset_index(drop=True)


        self.root = png_root
   
    def __getitem__(self, idx):
        image = np.asarray(Image.open(os.path.join(self.root, self.csv.loc[idx, 'filename_image'])).convert('L'))
        mask = np.asarray(Image.open(os.path.join(self.root, self.csv.loc[idx, 'filename_label'])).convert('L'))
        resolution = (int(self.csv.loc[idx, 'resolution'].split(',')[0][1:]),
                      int(self.csv.loc[idx, 'resolution'].split(',')[1][1:-1]))
        filename = self.csv.loc[idx, 'filename_image']
        aorta_visibility = self.csv.loc[idx, 'aorta_visibility']

        trans = transforms.Compose([transforms.ToTensor(),
                                    ])
        mask = trans(mask)
        image = trans(image)

        mask = TF.resize(mask, 224)
        image = TF.resize(image, 224)

        return {'image': image.float(),
                'mask': mask.long(),
                'resolution': torch.tensor(resolution),
                'aorta_visibility': aorta_visibility,
                'filename': filename}

    def __len__(self):
        return self.csv.shape[0]

In [ ]:
!git clone https://github.com/HuCaoFighting/Swin-Unet.git

Cloning into 'Swin-Unet'...
remote: Enumerating objects: 55, done.
remote: Total 55 (delta 0), reused 0 (delta 0), pack-reused 55
Unpacking objects: 100% (55/55), done.


In [ ]:
!pip install -q -r /content/Swin-Unet/requirements.txt

     |████████████████████████████████| 753.4 MB 7.3 kB/s 
     |████████████████████████████████| 4.0 MB 46.0 MB/s 
     |████████████████████████████████| 125 kB 69.7 MB/s 
     |████████████████████████████████| 77 kB 7.5 MB/s 
     |████████████████████████████████| 151 kB 65.1 MB/s 
     |████████████████████████████████| 48.4 MB 1.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.4.0 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.4.0 which is incompatible.


In [ ]:
!pip install -q -U torch torchvision

     |████████████████████████████████| 750.6 MB 11 kB/s 
     |████████████████████████████████| 21.0 MB 49.6 MB/s 


In [ ]:
!pip install -q einops timm yacs

     |████████████████████████████████| 431 kB 51.2 MB/s 


In [ ]:
%cd /content/Swin-Unet/

/content/Swin-Unet


In [ ]:
def iou_pytorch(outputs: torch.Tensor, labels: torch.Tensor):
    # You can comment out this line if you are passing tensors of equal shape
    # But if you are passing output from UNet or something it will most probably
    # be with the BATCH x 1 x H x W shape
    outputs = outputs.squeeze(1).byte()  # BATCH x 1 x H x W => BATCH x H x W
    labels = labels.squeeze(1).byte()
    SMOOTH = 1e-8
    intersection = (outputs & labels).float().sum((1, 2))  # Will be zero if Truth=0 or Prediction=0
    union = (outputs | labels).float().sum((1, 2))         # Will be zzero if both are 0
    
    iou = (intersection + SMOOTH) / (union + SMOOTH)  # We smooth our devision to avoid 0/0
    
    return iou

In [ ]:
import os
import yaml
from yacs.config import CfgNode as CN

_C = CN()

# Base config files
_C.BASE = ['']

# -----------------------------------------------------------------------------
# Data settings
# -----------------------------------------------------------------------------
_C.DATA = CN()
# Batch size for a single GPU, could be overwritten by command line argument
_C.DATA.BATCH_SIZE = 128
# Path to dataset, could be overwritten by command line argument
_C.DATA.DATA_PATH = ''
# Dataset name
_C.DATA.DATASET = 'imagenet'
# Input image size
_C.DATA.IMG_SIZE = 224
# Interpolation to resize image (random, bilinear, bicubic)
_C.DATA.INTERPOLATION = 'bicubic'
# Use zipped dataset instead of folder dataset
# could be overwritten by command line argument
_C.DATA.ZIP_MODE = False
# Cache Data in Memory, could be overwritten by command line argument
_C.DATA.CACHE_MODE = 'part'
# Pin CPU memory in DataLoader for more efficient (sometimes) transfer to GPU.
_C.DATA.PIN_MEMORY = True
# Number of data loading threads
_C.DATA.NUM_WORKERS = 8

# -----------------------------------------------------------------------------
# Model settings
# -----------------------------------------------------------------------------
_C.MODEL = CN()
# Model type
_C.MODEL.TYPE = 'swin'
# Model name
_C.MODEL.NAME = 'swin_tiny_patch4_window7_224'
# Checkpoint to resume, could be overwritten by command line argument
_C.MODEL.PRETRAIN_CKPT = '/content/swin_tiny_patch4_window7_224.pth'
_C.MODEL.RESUME = ''
# Number of classes, overwritten in data preparation
_C.MODEL.NUM_CLASSES = 2
# Dropout rate
_C.MODEL.DROP_RATE = 0.0
# Drop path rate
_C.MODEL.DROP_PATH_RATE = 0.1
# Label Smoothing
_C.MODEL.LABEL_SMOOTHING = 0.1

# Swin Transformer parameters
_C.MODEL.SWIN = CN()
_C.MODEL.SWIN.PATCH_SIZE = 4
_C.MODEL.SWIN.IN_CHANS = 3
_C.MODEL.SWIN.EMBED_DIM = 96
_C.MODEL.SWIN.DEPTHS = [2, 2, 6, 2]
_C.MODEL.SWIN.DECODER_DEPTHS = [2, 2, 6, 2]
_C.MODEL.SWIN.NUM_HEADS = [3, 6, 12, 24]
_C.MODEL.SWIN.WINDOW_SIZE = 7
_C.MODEL.SWIN.MLP_RATIO = 4.
_C.MODEL.SWIN.QKV_BIAS = True
_C.MODEL.SWIN.QK_SCALE = None
_C.MODEL.SWIN.APE = False
_C.MODEL.SWIN.PATCH_NORM = True
_C.MODEL.SWIN.FINAL_UPSAMPLE= "expand_first"

# -----------------------------------------------------------------------------
# Training settings
# -----------------------------------------------------------------------------
_C.TRAIN = CN()
_C.TRAIN.START_EPOCH = 0
_C.TRAIN.EPOCHS = 300
_C.TRAIN.WARMUP_EPOCHS = 20
_C.TRAIN.WEIGHT_DECAY = 0.05
_C.TRAIN.BASE_LR = 5e-4
_C.TRAIN.WARMUP_LR = 5e-7
_C.TRAIN.MIN_LR = 5e-6
# Clip gradient norm
_C.TRAIN.CLIP_GRAD = 5.0
# Auto resume from latest checkpoint
_C.TRAIN.AUTO_RESUME = True
# Gradient accumulation steps
# could be overwritten by command line argument
_C.TRAIN.ACCUMULATION_STEPS = 0
# Whether to use gradient checkpointing to save memory
# could be overwritten by command line argument
_C.TRAIN.USE_CHECKPOINT = False

# LR scheduler
_C.TRAIN.LR_SCHEDULER = CN()
_C.TRAIN.LR_SCHEDULER.NAME = 'cosine'
# Epoch interval to decay LR, used in StepLRScheduler
_C.TRAIN.LR_SCHEDULER.DECAY_EPOCHS = 30
# LR decay rate, used in StepLRScheduler
_C.TRAIN.LR_SCHEDULER.DECAY_RATE = 0.1

# Optimizer
_C.TRAIN.OPTIMIZER = CN()
_C.TRAIN.OPTIMIZER.NAME = 'adamw'
# Optimizer Epsilon
_C.TRAIN.OPTIMIZER.EPS = 1e-8
# Optimizer Betas
_C.TRAIN.OPTIMIZER.BETAS = (0.9, 0.999)
# SGD momentum
_C.TRAIN.OPTIMIZER.MOMENTUM = 0.9

# -----------------------------------------------------------------------------
# Augmentation settings
# -----------------------------------------------------------------------------
_C.AUG = CN()
# Color jitter factor
_C.AUG.COLOR_JITTER = 0.4
# Use AutoAugment policy. "v0" or "original"
_C.AUG.AUTO_AUGMENT = 'rand-m9-mstd0.5-inc1'
# Random erase prob
_C.AUG.REPROB = 0.25
# Random erase mode
_C.AUG.REMODE = 'pixel'
# Random erase count
_C.AUG.RECOUNT = 1
# Mixup alpha, mixup enabled if > 0
_C.AUG.MIXUP = 0.8
# Cutmix alpha, cutmix enabled if > 0
_C.AUG.CUTMIX = 1.0
# Cutmix min/max ratio, overrides alpha and enables cutmix if set
_C.AUG.CUTMIX_MINMAX = None
# Probability of performing mixup or cutmix when either/both is enabled
_C.AUG.MIXUP_PROB = 1.0
# Probability of switching to cutmix when both mixup and cutmix enabled
_C.AUG.MIXUP_SWITCH_PROB = 0.5
# How to apply mixup/cutmix params. Per "batch", "pair", or "elem"
_C.AUG.MIXUP_MODE = 'batch'

# -----------------------------------------------------------------------------
# Testing settings
# -----------------------------------------------------------------------------
_C.TEST = CN()
# Whether to use center crop when testing
_C.TEST.CROP = True

In [ ]:
import argparse
import logging
import os
import random
import numpy as np
from networks.vision_transformer import SwinUnet as ViT_seg

net = ViT_seg(_C, img_size=224, num_classes=2).cuda()
net.load_state_dict(torch.load('/content/swin_unet20epochs.pth'))

SwinTransformerSys expand initial----depths:[2, 2, 6, 2];depths_decoder:[1, 2, 2, 2];drop_path_rate:0.1;num_classes:2


/usr/local/lib/python3.7/dist-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


---final upsample expand_first---


<All keys matched successfully>

In [ ]:
resnet = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
resnet.fc = nn.Linear(512, 1)
resnet.load_state_dict(torch.load('/content/resnet18_v2.pth'))

# move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)
resnet.to(device)

net.eval()
resnet.eval()
for p in resnet.parameters():
    p.requires_grad = False

Downloading: "https://github.com/pytorch/vision/archive/v0.10.0.zip" to /root/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
!pip install simpleITK

In [ ]:
import SimpleITK as sitk
segthor_csv = pd.read_csv('/content/segthor.csv')

def reconstruct_volume(seg_model, cls_model, patient_id):
    slice_num = segthor_csv[segthor_csv['patient_ID'] == patient_id]['slice_num'].values[-1] + 1
    dataset = Patient('/content/segthor.csv', '/content/segthor/segthor png', patient_ID=patient_id)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=False)
    prediction = None
    with torch.no_grad():
        for data in dataloader:
            X_batch = data['image'].to(device)
            Y_batch = data['mask'].to(device)
            resolution = data['resolution'][0][0].item()

            cls_pred = (nn.Sigmoid()(cls_model(X_batch)) >= THRESHOLD) * 1

            cls_pred = cls_pred.squeeze(1)

            true_idxs = cls_pred.nonzero().squeeze(1).tolist()

            if len(true_idxs) == 0:
                Y_pred = torch.zeros((Y_batch.size(0), 2, Y_batch.size(2), Y_batch.size(3)), dtype=torch.float32).to(device)
            else:
                output = seg_model(X_batch[true_idxs]) 
                Y_pred = torch.zeros((Y_batch.size(0), 2, Y_batch.size(2), Y_batch.size(3)), dtype=torch.float32).to(device)
                Y_pred[true_idxs] = output

            Y_pred = interpolate(
                    Y_pred, size=(resolution, resolution), mode="bilinear", align_corners=False
                )
           
            if prediction is None:
                prediction = torch.clone(Y_pred.argmax(1))
            else:
                prediction = torch.cat((prediction, Y_pred.argmax(1)), dim=0)


        prd_itk = sitk.GetImageFromArray(prediction.detach().cpu().numpy().astype(np.uint8))
        prd_itk.SetSpacing((1, 1, 1))
        sitk.WriteImage(prd_itk, "pred.nii.gz") 

In [ ]:
reconstruct_volume(net, resnet, 1)

In [ ]:
def test_single_volume(image, label, net, classes, patch_size=[256, 256], test_save_path=None, case=None, z_spacing=1):
    image, label = image.squeeze(0).cpu().detach().numpy(), label.squeeze(0).cpu().detach().numpy()
    if len(image.shape) == 3:
        prediction = np.zeros_like(label)
        for ind in range(image.shape[0]):
            slice = image[ind, :, :]
            x, y = slice.shape[0], slice.shape[1]
            if x != patch_size[0] or y != patch_size[1]:
                slice = zoom(slice, (patch_size[0] / x, patch_size[1] / y), order=3)  # previous using 0
            input = torch.from_numpy(slice).unsqueeze(0).unsqueeze(0).float().cuda()
            net.eval()
            with torch.no_grad():
                outputs = net(input)
                out = torch.argmax(torch.softmax(outputs, dim=1), dim=1).squeeze(0)
                out = out.cpu().detach().numpy()
                if x != patch_size[0] or y != patch_size[1]:
                    pred = zoom(out, (x / patch_size[0], y / patch_size[1]), order=0)
                else:
                    pred = out
                prediction[ind] = pred
    else:
        input = torch.from_numpy(image).unsqueeze(
            0).unsqueeze(0).float().cuda()
        net.eval()
        with torch.no_grad():
            out = torch.argmax(torch.softmax(net(input), dim=1), dim=1).squeeze(0)
            prediction = out.cpu().detach().numpy()
    metric_list = []
    for i in range(1, classes):
        metric_list.append(calculate_metric_percase(prediction == i, label == i))

    if test_save_path is not None:
        img_itk = sitk.GetImageFromArray(image.astype(np.float32))
        prd_itk = sitk.GetImageFromArray(prediction.astype(np.float32))
        lab_itk = sitk.GetImageFromArray(label.astype(np.float32))
        img_itk.SetSpacing((1, 1, z_spacing))
        prd_itk.SetSpacing((1, 1, z_spacing))
        lab_itk.SetSpacing((1, 1, z_spacing))
        sitk.WriteImage(prd_itk, test_save_path + '/'+case + "_pred.nii.gz")
        sitk.WriteImage(img_itk, test_save_path + '/'+ case + "_img.nii.gz")
        sitk.WriteImage(lab_itk, test_save_path + '/'+ case + "_gt.nii.gz")
    return metric_list